In [ ]:
%pylab inline

In [ ]:
# Load LSTM network and generate text
import codecs
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "/home/jvdzwaan/data/tmp/lstm-alice/wonderland.txt"
with codecs.open(filename, 'r', encoding='utf-8') as f:
    raw_text = f.read().strip().lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "/home/jvdzwaan/data/tmp/2layers-alice/1.2272.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
def select_char(prediction, index_next, top=10):
    """Wanneer kies je voor het gegeven next character (uit de tekst) en wanneer kies je een ander char (op basis van de 
    prediction)? 
    Eerste idee: als index_next niet in de top 10 zit, dan sample je uit prediction, ander houd je index_next
    (of argmax uit prediction)
    """
    #print 'index next character:', index_next
    ind = numpy.argpartition(prediction, -top)[-top:]
    #print ind
    if index_next not in ind:
        #print 'sampling'
        idx = numpy.random.choice(n_vocab, p=prediction)
        #for i, j in enumerate(prediction):
        #    print '{}: {:.4f}'.format(i, j)
        print idx
        #return numpy.argmax(prediction)
        return numpy.random.choice(n_vocab, p=prediction)   
    return index_next

for i, j in enumerate(prediction[0]):
    print '{}: {:.4f}'.format(i, j)
select_char(prediction[0], 0)

In [ ]:
begin = [39, 21, 28, 28, 1, 39, 17, 25, 36, 6, 1, 17, 35, 1, 35, 24, 21, 1, 24, 17, 20, 1, 30, 31, 36, 24, 25, 30, 23, 1, 21, 28, 35, 21, 1, 36, 31, 1, 20, 31, 6, 1, 17, 30, 20, 0, 32, 21, 34, 24, 17, 32, 35, 1, 17, 22, 36, 21, 34, 1, 17, 28, 28, 1, 25, 36, 1, 29, 25, 23, 24, 36, 1, 36, 21, 28, 28, 1, 24, 21, 34, 1, 35, 31, 29, 21, 36, 24, 25, 30, 23, 1, 39, 31, 34, 36, 24, 1, 24, 21]
vervolg = u'aring. For some\nminutes it puffed away without speaking, but at last it unfolded its\narms, took the hookah out of its mouth again, and said, ‘So you think\nyou’re changed, do you?’'

In [ ]:
print len(begin)
matches = 0
t = []
print len(vervolg)
for c in vervolg.lower():
    #print c
    x = numpy.reshape(begin, (1, len(begin), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    
    index_next = char_to_int[c]
    result = select_char(prediction[0], index_next)
    
    begin.append(index_next)
    begin = begin[1:len(begin)]
    
    if int_to_char[result] == c:
        matches += 1
    t.append(int_to_char[result])
print matches
print ''.join(t)
print 
print vervolg

In [ ]:
int_to_char[6]

In [ ]:
import copy

seed = [39, 21, 28, 28, 1, 39, 17, 25, 36, 6, 1, 17, 35, 1, 35, 24, 21, 1, 24, 17, 20, 1, 30, 31, 36, 24, 25, 30, 23, 1, 21, 28, 35, 21, 1, 36, 31, 1, 20, 31, 6, 1, 17, 30, 20, 0, 32, 21, 34, 24, 17, 32, 35, 1, 17, 22, 36, 21, 34, 1, 17, 28, 28, 1, 25, 36, 1, 29, 25, 23, 24, 36, 1, 36, 21, 28, 28, 1, 24, 21, 34, 1, 35, 31, 29, 21, 36, 24, 25, 30, 23, 1, 39, 31, 34, 36, 24, 1, 24, 21]
vervolg = u'aring. For some\nminutes it puffed away without speaking, but at last it unfolded its\narms, took the hookah out of its mouth again, and said, ‘So you think\nyou’re changed, do you?’'

def generate(seed, vervolg, char_to_int, int_to_char):
    begin = copy.deepcopy(seed)
    #print len(begin)
    matches = 0
    t = []
    
    for c in vervolg.lower():
        x = numpy.reshape(begin, (1, len(begin), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
    
        index_next = char_to_int[c]
        result = select_char(prediction[0], index_next, top=1)
    
        begin.append(index_next)
        begin = begin[1:len(begin)]
    
        if int_to_char[result] == c:
            matches += 1
        t.append(int_to_char[result])
    print ''.join(t)

    return matches

#res = [generate(seed, vervolg, char_to_int, int_to_char) for i in range(100)]

In [ ]:
print res
print numpy.mean(res)
print numpy.std(res)
print numpy.min(res)
print numpy.max(res)
print numpy.mean(res)/len(vervolg) * 100

In [ ]:
seed = [1, 35, 24, 21, 1, 24, 17, 20, 0, 32, 28, 21, 30, 36, 41, 1, 31, 22, 1, 36, 25, 29, 21, 1, 17, 35, 1, 35, 24, 21, 1, 39, 21, 30, 36, 1, 20, 31, 39, 30, 1, 36, 31, 1, 28, 31, 31, 27, 1, 17, 18, 31, 37, 36, 1, 24, 21, 34, 1, 17, 30, 20, 1, 36, 31, 1, 39, 31, 30, 20, 21, 34, 1, 39, 24, 17, 36, 1, 39, 17, 35, 0, 23, 31, 25, 30, 23, 1, 36, 31, 1, 24, 17, 32, 32, 21, 30, 1, 30, 21]
vervolg = "xt. First, she tried to look down and make out what\nshe was coming to, but it was too dark to see anything; then she\nlooked at the sides of the well, and noticed that they were filled with\ncupboards and book-shelves; here and there she saw maps and pictures\nhung upon pegs. She took down a jar from one of the shelves as"

In [ ]:
print len(vervolg)
res = generate(seed, vervolg, char_to_int, int_to_char)
print res
print 
print vervolg

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
randn = np.random.randn
from pandas import *

idx = Index(arange(1,11))
df = DataFrame(randn(10, 5), index=idx, columns=['A', 'B', 'C', 'D', 'E'])
vals = np.around(df.values,2)
normal = plt.Normalize(vals.min()-1, vals.max()+1)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, frameon=True, xticks=[], yticks=[])

the_table=plt.table(cellText=vals, rowLabels=df.index, colLabels=df.columns, 
                    colWidths = [0.03]*vals.shape[1], loc='center', 
                    cellColours=plt.cm.RdYlGn(normal(vals)))

In [ ]:
def select_char_p(prediction, index_next, top=10):
    """Wanneer kies je voor het gegeven next character (uit de tekst) en wanneer kies je een ander char (op basis van de 
    prediction)? 
    Eerste idee: als index_next niet in de top 10 zit, dan sample je uit prediction, ander houd je index_next
    (of argmax uit prediction)
    """
    #print 'index next character:', index_next
    ind = numpy.argpartition(prediction, -top)[-top:]
    #print ind
    if index_next not in ind:
        print 'Changing!'
        #return numpy.argmax(prediction)
        idx = numpy.random.choice(n_vocab, p=prediction)
        return idx, prediction[idx]
    return index_next, prediction[index_next]

def color(vervolg, char_to_int, int_to_char):
    begin = [39, 21, 28, 28, 1, 39, 17, 25, 36, 6, 1, 17, 35, 1, 35, 24, 21, 1, 24, 17, 20, 1, 30, 31, 36, 24, 25, 30, 23, 1, 21, 28, 35, 21, 1, 36, 31, 1, 20, 31, 6, 1, 17, 30, 20, 0, 32, 21, 34, 24, 17, 32, 35, 1, 17, 22, 36, 21, 34, 1, 17, 28, 28, 1, 25, 36, 1, 29, 25, 23, 24, 36, 1, 36, 21, 28, 28, 1, 24, 21, 34, 1, 35, 31, 29, 21, 36, 24, 25, 30, 23, 1, 39, 31, 34, 36, 24, 1, 24, 21]
    #print len(begin)
    t = []
    p = []
    
    for c in vervolg.lower():
        x = numpy.reshape(begin, (1, len(begin), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
    
        index_next = char_to_int[c]
        real = prediction[0][index_next]
        result, pred = select_char_p(prediction[0], index_next)
        print result, pred, real
    
        begin.append(index_next)
        begin = begin[1:len(begin)]
    
        t.append(int_to_char[index_next])
        p.append(real)

    return ''.join(t), p
text, vals = color(vervolg, char_to_int, int_to_char)

In [ ]:
n=20
s = [text[i:i+n] for i in range(0, len(text), n)]
t = []
for l in s:
    t.append([c for c in l])
print t
print len(t)

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure()
ax = fig.add_subplot(111, xticks=[], yticks=[])

norm = plt.Normalize(0.0, 1.0)

nums = numpy.reshape(vals, (len(t), len(t[0])))

the_table=plt.table( cellText=t, cellColours=plt.cm.RdYlGn(norm(nums)), colWidths = [0.1]*nums.shape[1])

In [ ]:
print text

In [ ]:

a= numpy.reshape(vals, (len(t), len(t[0])))
print a.shape

In [ ]:
x = numpy.reshape(vals, (1, len(vals)))
x